In [1]:
import sys
sys.path.append("../")

In [10]:
import pandas as pd
import datetime as dt
from technical.indicators import rsi
from technical.udemy_patterns import apply_patterns

In [11]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE     
    
def apply_take_profit(row, PROFIT_FACTOR):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            if row.direction == BUY:
                return (row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
            else:
                return (row.ask_o - row.ask_c) * PROFIT_FACTOR + row.ask_o
        else:
            if row.direction == SELL:
                return (row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
            else:
                return (row.bid_o - row.bid_c) * PROFIT_FACTOR + row.bid_o
    else:
        return 0.0


def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            if row.direction == BUY:
                return row.ask_o
            else:
                return row.ask_c
        else:
            if row.direction == SELL:
                return row.bid_o
            else:
                return row.bid_c
    else:
        return 0.0

In [12]:
class Trade:
    def __init__(self, row, profit_factor, loss_factor):
        self.running = True
        self.start_index_m5 = row.name
        self.profit_factor = profit_factor
        self.loss_factor = loss_factor
        self.trigger_price = None
            
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, self.profit_factor, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, self.loss_factor, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, self.profit_factor, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, self.loss_factor, row.ask_h)

    def break_down_candle(self, row, buy=True):
        print('break_down_candle()')
        start_date = row.time
        end_date = start_date + dt.timedelta(minutes=55)
        DF_M5 = pd.read_pickle(f"../data/candles/EUR_USD_M5.pkl")
        df = DF_M5[(DF_M5.time >= start_date)&(DF_M5.time <= end_date)]
        for index, row in df.iterrows():
            if buy:
                if row.bid_h >= self.TP:
                    return 1, row.bid_h
                elif row.bid_l <= self.SL:
                    return 0, row.bid_l
            else:
                if row.ask_l <= self.TP:
                    return 1, row.ask_l
                elif row.ask_h >= self.SL:
                    return 0, row.ask_h
                
    def update_improved(self, row):
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP and row.bid_l <= self.SL:
                res, trigger_price = self.break_down_candle(row)
                if res:
                    self.close_trade(row, self.profit_factor, trigger_price)
                else:
                    self.close_trade(row, self.loss_factor, trigger_price)
            elif row.bid_h >= self.TP:
                self.close_trade(row, self.profit_factor, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, self.loss_factor, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP and row.ask_h >= self.SL:
                res, trigger_price = self.break_down_candle(row, buy=False)
                if res:
                    self.close_trade(row, self.profit_factor, trigger_price)
                else:
                    self.close_trade(row, self.loss_factor, trigger_price) 
            elif row.ask_l <= self.TP:
                self.close_trade(row, self.profit_factor, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, self.loss_factor, row.ask_h) 

In [13]:
def apply_signals(df, PROFIT_FACTOR, sig):
    df["SIGNAL"] = df.apply(sig, axis=1)
    df["TP"] = df.apply(apply_take_profit, axis=1, PROFIT_FACTOR=PROFIT_FACTOR)
    df["SL"] = df.apply(apply_stop_loss, axis=1)


def create_signals(df, time_d=1):
    df_signals = df[df.SIGNAL != NONE].copy() 
    df_signals['m5_start'] = [x + dt.timedelta(hours=time_d) for x in df_signals.time]
    df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'bid_o', 'bid_h', 'bid_l',
    'ask_o', 'ask_h', 'ask_l', 'direction'], axis=1, inplace=True)
    df_signals.rename(columns={
        'bid_c' : 'start_price_BUY', # for tracking the trade  
        'ask_c' : 'start_price_SELL',
        'm5_start' : 'time'
    }, inplace=True)
    return df_signals


class GuruTester:
    def __init__(self, df_big,
                    apply_signal, 
                    # df_m5,
                    LOSS_FACTOR = -1.0,
                    PROFIT_FACTOR = 1.5,
                    time_d=1 ):
        self.df_big = df_big.copy()
        self.apply_signal = apply_signal
        # self.df_m5 = df_m5.copy()
        self.LOSS_FACTOR = LOSS_FACTOR
        self.PROFIT_FACTOR = PROFIT_FACTOR
        self.time_d = time_d

        self.prepare_data()
        
    def prepare_data(self):
        
        print("prepare_data...")

        apply_signals(self.df_big,
                    self.PROFIT_FACTOR,
                    self.apply_signal)


        # df_m5_slim = self.df_m5[['time','bid_h', 'bid_l', 'ask_h', 'ask_l' ]].copy()
        # df_signals = create_signals(self.df_big, time_d=self.time_d)
        self.merged = self.df_big
        # self.merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')
        # self.merged.fillna(0, inplace=True)
        # self.merged.SIGNAL = self.merged.SIGNAL.astype(int)

    def run_test(self):
        print("run_test...")
        open_trades_m5 = []
        closed_trades_m5 = []

        for index, row in self.merged.iterrows():

            for ot in open_trades_m5:
                ot.update_improved(row)
                if ot.running == False:
                    closed_trades_m5.append(ot)
            
            if row.SIGNAL != NONE:
                open_trades_m5.append(Trade(row, self.PROFIT_FACTOR, self.LOSS_FACTOR))  
                
            
            open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

        self.df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5]) 
        print("Result:", self.df_results.result.sum())

In [14]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/candles/{pair}_H1.pkl")
    # df_m5 = pd.read_pickle(f"../data/candles/{pair}_M5.pkl")
    df_an = df_an[:1500].copy()
    
    df_an.reset_index(drop=True, inplace=True)
    # df_m5.reset_index(drop=True, inplace=True)
    df_an = rsi(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        # df_m5,
    )
    
    gt.run_test()
    return gt.df_results
    

In [15]:
res = []
for p in ["EUR_USD"]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
break_down_candle()
Result: -16.5


In [18]:
total_len = len(res[0]['res'])
bad_trades_len = len(res[0]['res'][res[0]['res'].result == -1.0])
total_len, bad_trades_len
print(f'{bad_trades_len} ({bad_trades_len / total_len * 100:.2f}%) wrong trades taken out of: {total_len} trades in total')

69 (66.35%) wrong trades taken out of: 104 trades in total


In [27]:
for r in res:
    print(r['pair'], r['res'].result.sum())

EUR_USD -1016.0
GBP_JPY -1057.5


In [ ]:
EUR_USD -1016.0
GBP_JPY -1057.5

In [7]:
type(DF_M5.iloc[1]['time'] - DF_M5.iloc[0]['time'])

pandas._libs.tslibs.timedeltas.Timedelta